In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load Boston Housing Authority data
rent_df = pd.read_excel("PS-Lookup-Tool-Eff-010124.xlsx", sheet_name="Sheet1")
# Average per bedroom, counting studios as 1 bed
rent_df['avg_per_bed'] = rent_df.iloc[:, 3:].apply(lambda x: x / int(x.name[1]) if x.name[1] != '0' else x).apply(
    lambda x: x.sum() / x.notnull().sum(), 1).sort_values()
rent_df = rent_df.sort_values('avg_per_bed')
# Save file
rent_df.to_csv("Rent_averages_BHA.tsv", sep='\t', index=None)
rent_df.to_csv("Rent_averages_BHA.csv", index=None)

In [5]:
rent_df.avg_per_bed.max()

1978.3526455026456

In [ ]:
# Load Zillow data
value_zip_df = pd.read_csv("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
rent_zip_df = pd.read_csv("Zip_zori_uc_sfrcondomfr_sm_month.csv")

# Load zoning compliance data
zone_df = pd.read_csv("parcel_zoning_compliance.csv", index_col=0)

# Get all zip codes from the parcel dataset
zip_l = [int(z[:4]) for z in zone_df['ZIP'].dropna().unique().astype(str) if (len(z[:4]) == 4) and ('.' not in z)]
full_zip_l = ['0'+z[:4] for z in zone_df['ZIP'].dropna().unique().astype(str) if (len(z[:4]) == 4) and ('.' not in z)]

# 1. Preliminary analysis/wrangling

In [47]:
# Compute the median rental price 
n_months = 12
rent_zip_df = rent_zip_df[rent_zip_df['Metro']=='Boston-Cambridge-Newton, MA-NH']
rent_zip_df[f'med_rent_past_{n_months}'] = rent_zip_df.apply(
    lambda x: x.iloc[-n_months:].sum() / x.iloc[-n_months:].notnull().sum(), 1)
rent_zip_df.sort_values('med_rent_past_12')

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29,med_rent_past_12
4673,59259,5601,3867,zip,NH,NH,Rochester,"Boston-Cambridge-Newton, MA-NH",Strafford County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1695.130009,1714.946979,1786.666667,1732.247885
4183,58506,4870,1832,zip,MA,MA,Haverhill,"Boston-Cambridge-Newton, MA-NH",Essex County,NaN,...,1872.798440,1899.395790,1877.679476,1857.500179,1837.408169,1871.482752,1922.703419,2007.731121,1946.555556,1886.911212
6388,59266,10704,3874,zip,NH,NH,Seabrook,"Boston-Cambridge-Newton, MA-NH",Rockingham County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1888.000000,1888.000000
5780,59236,7888,3842,zip,NH,NH,Hampton,"Boston-Cambridge-Newton, MA-NH",Rockingham County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1887.631525,1900.000000,1893.815762
2858,59218,3140,3820,zip,NH,NH,Dover,"Boston-Cambridge-Newton, MA-NH",Strafford County,NaN,...,1938.904302,2001.455954,2058.606454,2036.550677,1979.624522,1957.633607,2032.242310,2014.596400,1991.125000,1982.035348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,58758,3399,2446,zip,MA,MA,Brookline,"Boston-Cambridge-Newton, MA-NH",Norfolk County,2856.873545,...,3974.510070,3992.039261,3950.652795,3934.811368,3928.323057,4025.290155,4046.987967,4117.496350,4128.043612,3988.187935
4472,58536,5309,1890,zip,MA,MA,Winchester,"Boston-Cambridge-Newton, MA-NH",Middlesex County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4000.000000,4000.000000
3526,58632,3981,2118,zip,MA,MA,Boston,"Boston-Cambridge-Newton, MA-NH",Suffolk County,2903.723332,...,4028.816462,4087.373772,4100.384708,4109.713736,4105.202951,4111.724837,4096.114931,4081.290429,4063.804675,4064.686148
4681,58757,5612,2445,zip,MA,MA,Brookline,"Boston-Cambridge-Newton, MA-NH",Norfolk County,2843.899395,...,4066.500446,4079.960319,4046.379665,3994.412668,4032.782813,4056.355943,4131.472769,4217.625637,4305.373016,4089.913724


In [ ]:
# Compute median property value per allowed resident
# Subset for properties with a zoned density greater than 1 and properties with a value appraisal
per_resident_value_s = zone_df[(zone_df['zoned_density']>0) & (zone_df['TOTAL_VAL'].notnull())
                              ].groupby('municipal').apply(lambda x: (x['TOTAL_VAL'] / x['zoned_density']).median())
# Plot distribution
f, ax = plt.subplots(1, 1, figsize=(10,5))
ax.hist(per_resident_value_s, bins=np.linspace(1e4, 2e6, 20))
ax.set_title("Total value of properties per capita", size=16)
ax.set_xlabel("Per capita property value", size=14)
ax.set_ylabel("Frequency", size=14)
ax.get_xaxis().get_major_formatter().set_scientific(False)